# Web scraping av IMDb och skriv filmrecension

Hämta filmtitel, regissör, betyg, skådespelare med mera från Internet Movie Database och skriv en filmrecension som kan publiceras.

## Bibliotek och funktioner

`BeautifulSoup4` och `lxml` är två biblotek som används för att analysera HTML-sidor. Dessa kan behövas installeras med `pip` om de inte är installerade redan.

In [ ]:
# Installera bibliotek.
!pip install BeautifulSoup4
!pip install lxml

In [ ]:
import urllib.request
from lxml import html
from bs4 import BeautifulSoup 

# Funktion för stt skrapa HTML från webbsida.
def scrapewebpage(url):
	# Öppna URL och hämta HTML.
	web = urllib.request.urlopen(url)

	# Kontrollera att inga fel uppstod när URL öppnades.
	if (web.getcode() == 200):
		html = web.read()
		return(html)
	else:
		print("Error %s reading %s" % str(web.getcode()), url)

# Hjälpfunktion som skrapar webbsidor och gör en soppa av allt ihop.
def makesoup(url):
	html = scrapewebpage(url)
	return(BeautifulSoup(html, "lxml"))

## Skrapa Internet Movie Database

In [ ]:
# Skrapa Interstellar (2014).
soup = makesoup('http://www.imdb.com/title/tt0816692/')

In [ ]:
# Filmens ID.
movieid = soup.find(property="pageId").get("content")

# Filmens URL.
movielink = 'http://www.imdb.com/title/' + movieid + '/'

# Titel.
title = soup.find(itemprop="name").get_text()
title = title.strip() # Ta bort eventuella mellanslag före och efter texten.

# Årtal.
year = soup.find(id="titleYear").get_text()
year = year[1:5] # Ta bort parenteser, så att (2014) blir 2014.
year = int(year) # Gör om sträng till tal.

# Ta bort årtal från titel.
titleclean = title.replace("(" + str(year) + ")", "").strip()

# Längd.
duration = soup.find(itemprop="duration").get_text()
duration = duration.strip() # Ta bort eventuella mellanslag före och efter texten.

# Regissör.
director = soup.find(itemprop="director").get_text()
director = director.strip() # Ta bort eventuella mellanslag före och efter texten.

# Betyg.
rating = soup.find(itemprop="ratingValue").get_text()
rating = float(rating) # Gör om sträng till decimaltal (float).

# Skådespelare.
actors = []
for castlist in soup.find_all("table", "cast_list"):
	for actor in castlist.find_all(itemprop="actor"):
		actors.append(actor.get_text().strip())

## Presentera resultat

In [ ]:
print("Titel: " + titleclean)
print("År:    " + str(year))
print("Regi:  " + director)
print("Längd: " + duration)
print("Betyg: " + str(rating))
print("Skådespelare:")
for actor in actors:
	print("- " + actor)

## Skapa nyhetstext

In [ ]:
# Ju högre betyg, desto mer "imponerande" rubrik.
if rating == 10:
    newstitle = "Mästerverket {0} ({1}) av {2}".format(titleclean, year, director)
elif rating < 10 and rating > 8:
    newstitle = "Imponerande {0} ({1}) av {2}".format(titleclean, year, director)
elif rating <= 7 and rating > 4:
    newstitle = "{0} ({1}) av {2}".format(titleclean, year, director)
elif rating < 4:
    newstitle = "Bottennappet {0} ({1}) av {2}".format(titleclean, year, director)

In [ ]:
# Antal år sedan filmen slöppts på bio.
import datetime
now = datetime.datetime.now()

yearsago = now.year - year

In [ ]:
# Sätt ihop variablerna till brödtexten.
newsbody = """{0} av {1} producerades för {2} år sedan och har fått {3} i betyg på filmsajten IMDb.

I rollerna ser vi bland annat {4} och {5}.

{6}
""".format(titleclean, director, yearsago, rating, actors[0], actors[1], movielink)

## Visa nyhetstext

In [ ]:
print(newstitle)
print("-" * len(newstitle))
print(newsbody)